In [26]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
tavily_client = TavilyClient(api_key=tavily_key)
llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

In [2]:
google_agent=Google_agent()
deep_research_engine=Deep_research_engine()

In [3]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Deps:
    deep_research_output: dict

In [25]:
def google_agent_tool(query:str):
    """
    Use this tool to interact with the google agent, which can, search for images manage the user's calendar, emails, tasks, contacts, and more.
    Args:
        query (str): The entirequery related to the google agent and its capabilities
    Returns:
        str: The response from the google agent
    """
    return google_agent.chat(query)

async def deep_research_tool(ctx: RunContext[Deps], query:str):
    """
    Use this tool to do a deep research on a topic, to gather detailed informations and data.
    Args:
        query (str): The query related to the deep research and its capabilities
    Returns:
        str: The response from the deep research engine
    """
    response=await deep_research_engine.chat(query)
    ctx.deps.deep_research_output=response
    return response

async def quick_research_tool(query:str):
    """
    Use this tool to do a quick research on a topic, to gather basic informations and data.
    Args:
        query (str): the quick research query
    Returns:
        str: The response from the quick research engine
    """
    quick_research_agent=Agent(llm, tools=[tavily_search_tool(tavily_key)], system_prompt="do a websearch based on the query")
    result=await quick_research_agent.run(query)
    return result.data

def get_current_time_tool():
    """
    Use this tool to get the current time.
    Returns:
        str: The current time in a formatted string
    """
   
    return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"




In [27]:
class Cortana_agent:
    def __init__(self):
        self.agent=Agent(llm, tools=[google_agent_tool, deep_research_tool, quick_research_tool, get_current_time_tool], system_prompt="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you to help the user with their queries")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={})
    async def chat(self, query:str):
        result=await self.agent.run(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.data


In [28]:
cortana_agent=Cortana_agent()

In [37]:
print(await cortana_agent.chat('what is your name?'))

I'm Cortana, a helpful assistant.



In [38]:
cortana_agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are Cortana, a helpful assistant that can help with a wide range of tasks,                          you can use the tools provided to you to help the user with their queries', timestamp=datetime.datetime(2025, 4, 11, 20, 36, 35, 436342, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='what time is it?', timestamp=datetime.datetime(2025, 4, 11, 20, 36, 35, 436348, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='get_current_time_tool', args={}, tool_call_id='pyd_ai_a5d730889e4848b8bb676cb61b360ba0', part_kind='tool-call')], model_name='gemini-2.0-flash', timestamp=datetime.datetime(2025, 4, 11, 20, 36, 35, 995269, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='get_current_time_tool', content='The current time is 2025-04-11 16:36:35', tool_call_id='pyd_ai_a5d730889e48